# Week 5  Support Vector Machines  Kernel Trick  Regularization

In [ ]:

import pandas as pd
import numpy as np

# Paths to uploaded datasets
DATA_SMALL = "/mnt/data/T_ONTIME_REPORTING.csv"
DATA_LARGE = "/mnt/data/DelayData.csv"

# Utility: fast preview of a CSV
def fast_preview(path, n=5):
    print(f"Previewing {path}")
    df = pd.read_csv(path, nrows=n)
    display(df.head(n))
    return df

# Utility: chunked iterator for large CSV
def chunk_reader(path, chunksize=100_000, usecols=None, dtype=None):
    return pd.read_csv(path, chunksize=chunksize, usecols=usecols, dtype=dtype)

# Utility: downsample large dataset for experiments
def load_sample_from_large(n_rows=200_000, usecols=None):
    # Stream chunks until we accumulate n_rows
    rows = []
    total = 0
    for chunk in chunk_reader(DATA_LARGE, chunksize=100_000, usecols=usecols):
        rows.append(chunk)
        total += len(chunk)
        if total >= n_rows:
            break
    df = pd.concat(rows, ignore_index=True)
    print(f"Loaded sample of {len(df):,} rows from large file")
    return df

# Quick sanity check previews
_ = fast_preview(DATA_SMALL, n=5)
_ = fast_preview(DATA_LARGE, n=5)


In [ ]:

# Reuse the binary target from Week 4
usecols = [
    "arrdelay","depdelay","scheduledhour","month","dayofmonth",
    "temperature","windspeed","raindummy","snowdummy",
    "marketshareorigin","marketsharedest"
]
df = load_sample_from_large(n_rows=180_000, usecols=usecols).dropna()
df["target_delay15"] = (df["arrdelay"] > 15).astype(int)

X = df.drop(columns=["arrdelay","target_delay15"]).values
y = df["target_delay15"].values

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

pipe = make_pipeline(StandardScaler(), SVC(probability=True))

param_grid = {
    "svc__kernel": ["rbf","linear"],
    "svc__C": [0.1, 1, 10],
    "svc__gamma": ["scale", 0.1, 0.01]
}

grid = GridSearchCV(pipe, param_grid, cv=3, scoring="roc_auc", n_jobs=-1)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
best = grid.best_estimator_
proba = best.predict_proba(X_test)[:,1]
pred = (proba >= 0.5).astype(int)

print(classification_report(y_test, pred))
print("ROC AUC:", roc_auc_score(y_test, proba))


### Notes
Discuss kernel choice and regularization parameter C. Compare linear with RBF. Report ROC AUC and confusion matrix insights.